Work on ace exceptions here. 

Edge cases and special behaviours related to splitting.

* Split aces - no hitting and no doubling; i.e. only can "stick". 

* At what level should this be included? Inside main decision module (basic_player_multiple_hits) or as part of CF within player_multiple_turns?
* Decided to place it at level of CF within player_multiple_turns as it is a minority case/exception behaviour that only occurs within context of splitting. This can also cover resplitting aces; we check the hand and see if there is only one ace. 

* Resplitting aces - you can resplit aces.

* Split aces and blackjack - you can only get a standard 21 after you have split aces, and cannot blackjack - no need for blackjack detector.

* As playerblackjack indicator is only set during openinghand(); it will be set to zero by default. And also if it was set to 1 in the openinghand(), we wouldn't get to the splitting stage. 


* Exception thrown when one of the subhands is of the blackjack variety i.e. hand = [A, K/Q/J/10] and corresponding permutat. Using the existing basic_player_multiple_hits decision module; as player_blackjack variable is set during opening hands, the decision module will treat this hand as a soft total and perform a lookup, and the BS strategy table will NOT contain any of AJ, AQ, AK (although A10 will be there as it was placed there to deal with three card soft totals).


* Design consideration: Either amend the decision module; or the basic strategy table; or the CF inside player_multiple_turn()
module. Opted for the last option, but it is currently looking a little ugly - lots of ifs.

* Exception thrown when splitting one hand e.g. [A, A], yields TWO further [A, A], [A, A] hands.


* Pathological behaviour 1 - 2 pairs occurring after 1st decision-action sequence in for loop:


* Original multiwaysplit code was written assuming that only one of the new hands would have a pair, NOT both.

* Original player_multiple_turn() logic: 

* Assume as usual that player_action_parent() has already produced a list of lists and populated it with cards; so we start with player_split_hands = [[A, 2], [A, A]].
* Element[0] will be resolved in 1st iteration.
* Then we get to element[1]
* decision = {Sp}
* Assuming next two cards are A, A, then new player_split_hands = [[A, 2], [A, A], [A, A]] and new hands [A, A] and [A, A]
* player = player_split_hands[1] = [A, A]
* IF conditional will be triggered as decision = {Sp}
* Then decision = basic_player_multiple_hits(player, dealer, player_blackjack) = {Sp}
* Assuming new cards are 4, 9, then new player split hands will be = [[A, 2], [A, 4], [A, 9], [A, A]] and new hands [A, 4], [A,9]
* player = player_split_hands[1] = [A, 4]
* BUT THEN this will be passed to playerturn(), which will recommend H, and so the force decision - {S} will be missed, card will be drawn and error.


* Pathological behaviour 2 - indefinite numbers of pairs occuring after 1st decision-action sequence. 

*Reverted back to multiwaysplit(1) if conditional; with the minor amendments for ace exceptions. Works fine and seems to be consistent with final output for multiwaysplits(1), but you need to double check. 


* Pathological behaviour 3 - making above handling consistent with non-pathological behaviours.

* Clarify issue precisely - debrief and write up written notes.


* For loop and function that can handle spectrum of split behaviours has been written up. Now for the ace-related exceptions.

In [47]:
import pandas as pd
import random

columns = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
index = [i for i in range(5, 22)]
index2 = ['A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10']
index3 = ['2, 2','3, 3', '4, 4', '5, 5', '6, 6', '7, 7', '8, 8', '9, 9', '10, 10', 'J, J', 'Q, Q', 'K, K', 'A, A']
indices = index + index2 + index3
basic_strategy_table = pd.read_csv('basic-strategy.csv', header=None)
basic_strategy_table.index = indices
basic_strategy_table.columns = columns

basic_strategy_table_no_doubling = pd.read_csv('basic-strategy-no-doubling.csv', header=None)
basic_strategy_table_no_doubling.index = indices
basic_strategy_table_no_doubling.columns = columns

In [48]:
def getcards(decks=1):
    """Brings out a number of sealed decks of cards in their original order to the virutal gaming table.

    Creates a dictionary tracking the name of each card, its quantity, and its value
    
    Keyword argument:
        decks {int} -- the number of 52-card decks that the casino is using (default 1)
    
    Returns:
        dictionary{keys: values} -- dictionary object where keys are the card name (e.g. 'K' for King )
                                    and where the values are a list object containing the card's quantity and its value.
                                    
    The value for the ace contains three elements, as aces can further take on the value of 1 or 11, depending on 
    the context of the hand. No distinction is made between card suits, i.e. spades, hearts, clubs, diamonds.
    """
    
    deck = {}
    total_cards = decks * 52
    card_quantity = int(total_cards / 13)
    
    deck['A'] = [card_quantity, 1, 11]

    for card in range(2, 11):
        deck[str(card)] = [card_quantity, card]
    
    for card in "JQK":
        deck[str(card)] = [card_quantity, 10]
        
    return deck, total_cards

deck, total_cards = getcards(decks=1)

In [49]:
def shuffler(deck, total_cards):
    
    """Shuffles the decks of cards, and places them inside the virtual gaming table card-shoe."""
    
    shuffled_shoe = []
    
    while len(shuffled_shoe) < total_cards:
        draw = random.choice(list(deck))
        if deck[draw][0] == 0:
            pass
        else:
            deck[draw][0] -= 1
            shuffled_shoe.append(draw)
            
    return shuffled_shoe

def openinghand():
    
    """Deals two cards to the player, and one to the dealer from the shuffled shoe. Outputs a message if there is Blackjack
    
    Cards are taken sequentially from the shuffled card shoe, beginning with the last element of the shuffled shoe list object
    and working in reverse order."""
    
    player = []
    dealer = []

    player.append(shuffled_shoe.pop())
    player.append(shuffled_shoe.pop())
    
    dealer.append(shuffled_shoe.pop())
    
    if ('A' in player) and (('10' in player) or ('J' in player) or ('Q' in player) or ('K' in player)):
        player_blackjack = 1
        print("Blackjack, house pays out at 3:2")
    else:
        player_blackjack = 0
        print("No Blackjack this time")
        
    return player, dealer, player_blackjack

def decode(action):
    if action == 'Sp':
        message = 'split'
    elif action == 'H':
        message = 'hit'
    elif action == 'D':
        message = 'double down'
    else:
        message = 'stick'
    return message

def basic_player_multiple_hits(player, dealer, player_blackjack):
    
    if len(player) == 2:
        
        if player_blackjack == 1:
            cardvalues = [21]
            decision = 'S'
            print("It's my lucky day")
        else:
            if player[0] == player[1]:
                lookup_pairs = player[0] + ", " + player[1]
                decision = basic_strategy_table.loc[lookup_pairs, dealer[0]]
                print("As I have a pair of {}s, I am going to {}".format(player[0], decode(decision)))
            elif ('A' in player):
                if player[0] == 'A':
                    lookup_soft_total = player[0] + player[1]
                else:
                    lookup_soft_total = player[1] + player[0]
                
                decision = basic_strategy_table.loc[lookup_soft_total, dealer[0]]
                print("As I have a soft total, I am going to {}".format(decode(decision)))
            else:
                cardvalues = []
                for card in player:
                    cardvalues.append(deck[card][1])
                
                lookup_hard_total = sum(cardvalues)
                decision = basic_strategy_table.loc[lookup_hard_total, dealer[0]]
                print("I have a total of {}, I am going to {}".format(lookup_hard_total, decode(decision)))
        
    if len(player) > 2: 
        
        if 'A' in player:  # check if the >3 hand has an ace, i.e. is potentially soft.
            if player[0] != 'A':  # re-order dual ace to the left
                ace_index = player.index('A')  
                player[0], player[ace_index] = player[ace_index], player[0]
            
            cardvalues_no_ace = [deck[card][1] for card in player[1::]]
            
            if sum(cardvalues_no_ace) <= 10: # check soft totals
                lookup_soft_total = player[0] + str(sum(cardvalues_no_ace))

                decision = basic_strategy_table_no_doubling.loc[lookup_soft_total, dealer[0]]
                print("As I have a soft {}, that is {} or {}, I am going to {}.".format(11 + sum(cardvalues_no_ace), 1 + sum(cardvalues_no_ace), 11 + sum(cardvalues_no_ace), decode(decision)))
            else:
                lookup_hard_total = deck['A'][1] + sum(cardvalues_no_ace) # variable naming, hard total as ace can only take value 1 now
                
                decision = basic_strategy_table_no_doubling.loc[lookup_hard_total, dealer[0]]
                print("I have {}, so I am going to {}".format(1 + sum(cardvalues_no_ace), decode(decision)))
        else:
            cardvalues = [deck[card][1] for card in player]
            lookup_hard_total = sum(cardvalues)
            
            decision = basic_strategy_table_no_doubling.loc[lookup_hard_total, dealer[0]]
            print("I have {}, so I am going to {}".format(sum(cardvalues), decode(decision)))
            
    return decision

In [50]:
def player_action(player, decision):
    
    if decision == 'S':
        print(player)
        return player# -> go to dealerturn()

    if decision == 'D':
        drawcard = shuffled_shoe.pop()
        player.append(drawcard)
        print(player) #-> go to dealerturn(), and also have to modify payout
        return player
        
    
    if decision == 'H':
        drawcard = shuffled_shoe.pop()
        player.append(drawcard)
        print(player) #-> now there are three cards in player hand and one in dealer
        return player
        

    if decision == 'Sp': 
        multiple_hand_player = [player[i:i+1] for i in range(len(player))] #splits the player hand into two cards and places them in two sub-hands
    
        drawcard1 = shuffled_shoe.pop()
        drawcard2 = shuffled_shoe.pop()
    
        print("After, splitting, I receive {} and {} in each of my hands".format(drawcard1, drawcard2))
    
        multiple_hand_player[0].append(drawcard1)
        multiple_hand_player[1].append(drawcard2)
        
        print("My new hands are: " + str(multiple_hand_player))
        
        return multiple_hand_player

In [51]:
def player_turn(player, decision):

    while decision == 'H':
        playerbust_indicator = check_if_bust(player)
        
        if playerbust_indicator == 0:
            print("Continue decision-action sequence")
            decision = basic_player_multiple_hits(player, dealer, player_blackjack)
            print(decision)
            player = player_action(player, decision)
            print(player)
        else:
            print("Now dealer's turn as I have gone bust, CF takes over")
            break
    else:
        if decision == 'S':
            print("Decision-action sequence terminated by decision = 'S'")
            print("Go to dealer, CF takes over")
            playerbust_indicator = 0
        elif decision == 'D':
            print("Decision-action sequence terminated by decision = 'D'")
            print("Go to dealer, CF takes over")
            playerbust_indicator = check_if_bust(player)
            
            if playerbust_indicator:
                print("Player doubled and busted.")
            else:
                print("Player doubled and did not bust.")
                
        elif decision == 'Sp':  # This part was added hastily - you need to think out the logic of this through.
            playerbust_indicator = check_if_bust(player)
        
    return player, decision, playerbust_indicator

In [52]:
def check_if_bust(player):
    if 'A' in player:
        if player[0] != 'A':
            ace_index = player.index('A')
            player[0], player[ace_index] = player[ace_index], player[0]
            
        card_values_no_ace = [deck[card][1] for card in player[1::]]
        
        if sum(card_values_no_ace) <= 10:
            bust_indicator = 0
        elif sum(card_values_no_ace) > 20:
            bust_indicator = 1
        else:
            bust_indicator = 0
    else:
        card_values = [deck[card][1] for card in player]
        
        if sum(card_values) > 21:
            bust_indicator = 1
        else:
            bust_indicator = 0
            
    return bust_indicator

In [53]:
def player_action_parent(player, decision):
    
    if decision == 'S':
        print(player)
        return player# -> go to dealerturn()

    if decision == 'D':
        drawcard = shuffled_shoe.pop()
        player.append(drawcard)
        print(player) #-> go to dealerturn(), and also have to modify payout
        return player
        
    
    if decision == 'H':
        drawcard = shuffled_shoe.pop()
        player.append(drawcard)
        print(player) #-> now there are three cards in player hand and one in dealer
        return player
        

    if decision == 'Sp': 
        player_split_hands = [player[i:i+1] for i in range(len(player))] #splits the player hand into two cards and places them in two sub-hands
    
        drawcard1 = shuffled_shoe.pop()
        drawcard2 = shuffled_shoe.pop()
                    
        print("After, splitting, I receive {} and {} in each of my hands".format(drawcard1, drawcard2))
    
        player_split_hands[0].append(drawcard1)
        player_split_hands[1].append(drawcard2)
        
        if [hand[0] == hand[1] for hand in player_split_hands] == [True, False]:  # if additional pairs are drawn then these are placed at the end of the list of lists.
            player_split_hands[0], player_split_hands[1] = player_split_hands[1], player_split_hands[0]
        
        print("My new hands are: " + str(player_split_hands))
        
        return player_split_hands

In [54]:
def player_action_child(player, decision):
    
    if decision == 'S':
        print(player)
        return player# -> go to dealerturn()

    if decision == 'D':
        drawcard = shuffled_shoe.pop()
        player.append(drawcard)
        print(player) #-> go to dealerturn(), and also have to modify payout
        return player
        
    
    if decision == 'H':
        drawcard = shuffled_shoe.pop()
        player.append(drawcard)
        print(player) #-> now there are three cards in player hand and one in dealer
        return player  

    if decision == 'Sp':
        
        index = player_split_hands.index(player) 
        
        player_split_hands.insert(index + 1, list(player.pop()))  # Turn one hand with a pair into two separate hands inside player_split_hands
    
        drawcard1 = shuffled_shoe.pop()
        drawcard2 = shuffled_shoe.pop()
    
        print("After, splitting, I receive {} and {} in each of my hands".format(drawcard1, drawcard2))
    
        player_split_hands[index].append(drawcard1)
        player_split_hands[index + 1].append(drawcard2)
        
        if [hand[0] == hand[1] for hand in player_split_hands[index:index + 2]] == [True, False]: # Ensures that if further pairs are drawn these are not resolved first.
            player_split_hands[index], player_split_hands[index + 1] = player_split_hands[index + 1], player_split_hands[index]
        
        print("My new hands are: {} and {}.".format(player_split_hands[index], player_split_hands[index + 1]))
        
        player = player_split_hands[index] 
        
        return player

In [ ]:
# Full module including ace-related exception behaviours. 

def player_multiple_turns(player_split_hands):
    
    decision_list = []
    playerbust_indicator_list = []

    for hand in player_split_hands:
    
        if ('A' in hand) and (('10' in hand) or ('J' in hand) or ('Q' in hand) or ('K' in hand)):
            decision = 'S'
        else:
            decision = basic_player_multiple_hits(hand, dealer, player_blackjack)
            
        if (hand[0] == 'A') and (hand[1] != 'A'):
            decision = 'S'
        player = player_action_child(hand, decision)
    
        if decision == 'Sp':
            while player[0] == player[1]:
                decision = basic_player_multiple_hits(player, dealer, player_blackjack)
                player = player_action_child(player, decision)
            else:  
                if ('A' in player) and (('10' in player) or ('J' in player) or ('Q' in player) or ('K' in player)): 
                    decision = 'S'
                else:
                    decision = basic_player_multiple_hits(player, dealer, player_blackjack)
            
                if (player[0] == 'A') and (player[1] != 'A'): 
                    decision = 'S'
                player = player_action_child(player, decision)
            
        player, decision, playerbust_indicator = player_turn(player, decision)
        decision_list.append(decision)
        playerbust_indicator_list.append(playerbust_indicator)
    
    return player_split_hands, decision_list, playerbust_indicator_list

In [ ]:
# For loop that I am fairly confident can deal with all tested pathological behaviour and
# ace exceptions.

decision_list = []
playerbust_indicator_list = []

for hand in player_split_hands:
    
    print("Loop is working on hand: {}".format(hand))
    
    if ('A' in hand) and (('10' in hand) or ('J' in hand) or ('Q' in hand) or ('K' in hand)): # prevents this being treated as a soft-total.
        decision = 'S'
    else:
        decision = basic_player_multiple_hits(hand, dealer, player_blackjack)
        print("Looking up decision for hand {} in lookup table".format(hand))
            
    if (hand[0] == 'A') and (hand[1] != 'A'): # Enforces one card only after having split aces. But does allow resplitting aces.
        decision = 'S'
        print("Player cannot draw any more cards having split aces.")
    player = player_action_child(hand, decision)
    
    print("Carrying out action on hand according to decision {}.".format(decision))
    
    # THis will run as long as there either further splits to be made (through decision or by composition of player hand)
    # We do not want further decision-action sequences to be called, nor test the condition in the while loop if there is
    # no need to do so. 
    
    if decision == 'Sp':
        while player[0] == player[1]:
            print("Loop initiated")
            decision = basic_player_multiple_hits(player, dealer, player_blackjack)
            player = player_action_child(player, decision)
        else:
            print("Loop terminated due to a non-pair hand, {} being drawn and indexed to player".format(player))
            print("Now resolving this hand with decision-action sequence.")
            
            if ('A' in player) and (('10' in player) or ('J' in player) or ('Q' in player) or ('K' in player)): 
                decision = 'S'
            else:
                decision = basic_player_multiple_hits(player, dealer, player_blackjack)
            
            if (player[0] == 'A') and (player[1] != 'A'): 
                decision = 'S'
                print("Player cannot draw any more cards having split aces")
            player = player_action_child(player, decision)
            
    ## In case that there are no further splits (case). Can I be sure the next line will execute even after else has been executed?
    player, decision, playerbust_indicator = player_turn(player, decision)
    decision_list.append(decision)
    playerbust_indicator_list.append(playerbust_indicator)
    
    print(player_split_hands)

In [ ]:
# Function module without ace related exception behaviours.

def player_multiple_turns(player_split_hands):

    decision_list = []
    playerbust_indicator_list = []

    for hand in player_split_hands:
    
        decision = basic_player_multiple_hits(hand, dealer, player_blackjack)
        player = player_action_child(hand, decision)
    
        if decision == 'Sp':
            while player[0] == player[1]:
                print("Loop initiated")
                decision = basic_player_multiple_hits(player, dealer, player_blackjack)
                player = player_action_child(player, decision)
            else:
                print("Loop terminated due to a non-pair hand, {} being drawn and indexed to player".format(player))
                print("Now resolving this hand with decision-action sequence.")
                decision = basic_player_multiple_hits(player, dealer, player_blackjack)
                player = player_action_child(player, decision)
            
        player, decision, playerbust_indicator = player_turn(player, decision)
        decision_list.append(decision)
        playerbust_indicator_list.append(playerbust_indicator)
    
        print(player_split_hands)
        
    return player_split_hands, decision_list, playerbust_indicator_list

In [ ]:
# For loop for function module without ace related exception behaviours.

decision_list = []
playerbust_indicator_list = []

for hand in player_split_hands:
    
    print("Loop is working on hand: {}".format(hand)) 
    
    decision = basic_player_multiple_hits(hand, dealer, player_blackjack)
    
    print("Looking up decision for hand {} in lookup table".format(hand))
    player = player_action_child(hand, decision)
    
    print("Carrying out action on hand according to decision {}.".format(decision))
    
    # THis will run as long as there either further splits to be made (through decision or by composition of player hand)
    # We do not want further decision-action sequences to be called, nor test the condition in the while loop if there is
    # no need to do so. 
    
    if decision == 'Sp':
        while player[0] == player[1]:
            print("Loop initiated")
            decision = basic_player_multiple_hits(player, dealer, player_blackjack)
            player = player_action_child(player, decision)
        else:
            print("Loop terminated due to a non-pair hand, {} being drawn and indexed to player".format(player))
            print("Now resolving this hand with decision-action sequence.")
            decision = basic_player_multiple_hits(player, dealer, player_blackjack)
            player = player_action_child(player, decision)
            
    ## In case that there are no further splits (case). Can I be sure the next line will execute even after else has been executed?
    player, decision, playerbust_indicator = player_turn(player, decision)
    decision_list.append(decision)
    playerbust_indicator_list.append(playerbust_indicator)
    
    print(player_split_hands)

In [147]:
# Testing ace behaviour - working with function module

rigged_shoe = []

for card in "9322A4A95AA7A2AAA2A77AA":
    rigged_shoe.append(card)
    
print(rigged_shoe)

['9', '3', '2', '2', 'A', '4', 'A', '9', '5', 'A', 'A', '7', 'A', '2', 'A', 'A', 'A', '2', 'A', '7', '7', 'A', 'A']


In [129]:
# Testing for multi-way splits - working with function module

rigged_shoe = []

for card in "9999992A4A95AA7A27A7277277":
    rigged_shoe.append(card)
    
print(rigged_shoe)

['9', '9', '9', '9', '9', '9', '2', 'A', '4', 'A', '9', '5', 'A', 'A', '7', 'A', '2', '7', 'A', '7', '2', '7', '7', '2', '7', '7']


In [141]:
# Testing pathological behaviour level #1 and #2 - two pairs drawn after splitting the iteration hand, and indefinite
# number of pair hands drawn after 1st decision-action sequence. 

# Working with function module.

rigged_shoe = []

for card in "2222229999992A4A977777477277777597A7777277":
    rigged_shoe.append(card)
    
print(rigged_shoe)

['2', '2', '2', '2', '2', '2', '9', '9', '9', '9', '9', '9', '2', 'A', '4', 'A', '9', '7', '7', '7', '7', '7', '4', '7', '7', '2', '7', '7', '7', '7', '7', '5', '9', '7', 'A', '7', '7', '7', '7', '2', '7', '7']


In [153]:
# Testing to make the module work for when there are no splits.

rigged_shoe = []

for card in "2222229999992A4A977777477277777597A7772223":
    rigged_shoe.append(card)
    
print(rigged_shoe)

['2', '2', '2', '2', '2', '2', '9', '9', '9', '9', '9', '9', '2', 'A', '4', 'A', '9', '7', '7', '7', '7', '7', '4', '7', '7', '2', '7', '7', '7', '7', '7', '5', '9', '7', 'A', '7', '7', '7', '2', '2', '2', '3']


In [182]:
deck, total_cards = getcards(decks=20)
shuffled_shoe = shuffler(deck, total_cards)

In [183]:
dealer

['5']

In [220]:
player, dealer, player_blackjack = openinghand()
decision = basic_player_multiple_hits(player, dealer, player_blackjack)
player_split_hands = player_action_parent(player, decision)

No Blackjack this time
I have a total of 8, I am going to hit
['6', '2', 'A']


In [221]:
# sporadic test

player_split_hands = [['8', '7'], ['A', 'A']]
player_multiple_turns(player_split_hands)

I have a total of 15, I am going to hit
['8', '7', '10']
Now dealer's turn as I have gone bust, CF takes over
As I have a pair of As, I am going to split
After, splitting, I receive 2 and 10 in each of my hands
My new hands are: ['A', '2'] and ['A', '10'].
As I have a soft total, I am going to hit
['A', '2']
Decision-action sequence terminated by decision = 'S'
Go to dealer, CF takes over
['A', '10']
Decision-action sequence terminated by decision = 'S'
Go to dealer, CF takes over


([['8', '7', '10'], ['A', '2'], ['A', '10']], ['H', 'S', 'S'], [1, 0, 0])

In [205]:
player_split_hands, decision_list, playerbust_indicator_list = player_multiple_turns(player_split_hands)

I have 19, so I am going to stick
['8', '7', '4']
Decision-action sequence terminated by decision = 'S'
Go to dealer, CF takes over


KeyError: 22

In [152]:
print(player_split_hands, decision_list, playerbust_indicator_list)

[['A', '7'], ['A', '2'], ['A', '2'], ['A', '7'], ['A', '5'], ['A', '9'], ['A', '4'], ['A', '2'], ['A', '2'], ['A', '3'], ['A', '9']] ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


* Pathological for loop and also ace-related behaviours before turning into function:



* Below are obsolete modules that were not able to deal with errors 1 and 2, but could handle ace related exception behaviours.

In [ ]:
# Process: If you split aces, then dealer will only give you one card for each new hand.
# No hitting, no doubling, but further splitting is allowed if you happen to draw another ace.


def player_multiple_turns(player_split_hands):
    
    decision_list = []
    playerbust_indicator_list = []
    
    for hand in player_split_hands:
        
        print("Loop is working on hand: {}".format(hand)) 
        if ('A' in hand) and (('10' in hand) or ('J' in hand) or ('Q' in hand) or ('K' in hand)):
            decision = 'S'
        else:
            decision = basic_player_multiple_hits(hand, dealer, player_blackjack)
        if hand[0] == 'A':
            decision = 'S'
            print("Player cannot draw any more cards having split aces.")           
        player = player_action_child(hand, decision)
        
        if decision == 'Sp': 
            print("Loop entered due to another split in the subhand encountered")
            if ('A' in player) and (('10' in player) or ('J' in player) or ('Q' in player) or ('K' in player)):
                decision = 'S'
            else:
                decision = basic_player_multiple_hits(player, dealer, player_blackjack) 
            if player[0] == 'A':
                decision = 'S'
                print("Player cannot draw any more cards having split aces.") 
            player = player_action_child(player, decision)    
        
        player, decision, playerbust_indicator = player_turn(player, decision)
        
        print("The decision that is appended to the decision list is {}.".format(decision))
        
        decision_list.append(decision)
        playerbust_indicator_list.append(playerbust_indicator)
        
        print("The list of lists is {}".format(player_split_hands))
    
    return player_split_hands, decision_list, playerbust_indicator_list

In [ ]:
# Process: If you split aces, then dealer will only give you one card for each new hand.
# No hitting, no doubling, but further splitting is allowed if you happen to draw another ace.


def player_multiple_turns(player_split_hands):
    
    decision_list = []
    playerbust_indicator_list = []
    
    for hand in player_split_hands:
        
        print("Loop is working on hand: {}".format(hand)) 
        if ('A' in hand) and (('10' in hand) or ('J' in hand) or ('Q' in hand) or ('K' in hand)):
            decision = 'S'
        else:
            decision = basic_player_multiple_hits(hand, dealer, player_blackjack)
        if hand[0] == 'A':
            decision = 'S'
            print("Player cannot draw any more cards having split aces.")           
        player = player_action_child(hand, decision)
        
        if decision == 'Sp': 
            print("Loop entered due to another split in the subhand encountered")
            if ('A' in player) and (('10' in player) or ('J' in player) or ('Q' in player) or ('K' in player)):
                decision = 'S'
            else:
                decision = basic_player_multiple_hits(player, dealer, player_blackjack) 
            if player[0] == 'A':
                decision = 'S'
                print("Player cannot draw any more cards having split aces.") 
            player = player_action_child(player, decision)    
        
        player, decision, playerbust_indicator = player_turn(player, decision)
        
        print("The decision that is appended to the decision list is {}.".format(decision))
        
        decision_list.append(decision)
        playerbust_indicator_list.append(playerbust_indicator)
        
        print("The list of lists is {}".format(player_split_hands))
    
    return player_split_hands, decision_list, playerbust_indicator_list

In [ ]:
# If player has only one ace in new split hands, he must stick, and cannot
# hit nor double. Corresponds to only receiving one card after splitting aces.
# However splitting aces is still possible.
# Interrupts CF within player_multiple_turns and forces his decision.

# A little concerned about local vs nonlocal, global variable declaration
# and unexpected behaviours. So this was not used.

# Instead hard coded some more conditionals in terms of hand and player
# inside for loop iterator.

# Come back to this as an optimisation point. Hmm. Maybe no need with new amendment.


def split_aces_one_card(player, decision):
    
    if player.count('A') == 1:
        decision = 'S'
        print("Player cannot draw anymore cards after split aces.")
    else:
        decision = decision 
        
    return decision

In [ ]:
def split_aces_no_blackjack(player):
    
    if ('A' in player) and (('10' in player) or ('J' in player) or ('Q' in player) or ('K' in player)):
        decision = 'S'
    else:
        decision = "unassigned variable placeholder that will be reassigned by basic_player_multiple_hits function "
    
    return decision